In [1]:
from acne_detection import AcneDetector
from age_gender_estimation import AgeGenderEstimator
from dark_eye_detection import DarkEyeDetector
from facial_landmark import FacialLandmark
from wrinkles_detection import WrinklesDetector
from recommendation_system import ProductRecommendation
from sallowness_detection import get_sallowness_score
from os.path import join
import pandas as pd
import config
import json
import random
import time
import imutils

c:\users\denis\anaconda3\envs\fyp\lib\site-packages\cntk\cntk_py_init.py:84: UserWarning: 

################################################ Missing optional dependency (GPU-Specific) ################################################
   CNTK may crash if the component that depends on those dependencies is loaded.
   Visit https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Windows-Python#optional-gpu-specific-packages for more information.
############################################################################################################################################
If you intend to use CNTK without GPU support, you can ignore the (likely) GPU-specific warning!
############################################################################################################################################

  warnings.warn(WARNING_MSG_GPU_ONLY % ('GPU-Specific', 'https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Windows-Python#optional-gpu-specific-packages'))
[nltk_dat

In [7]:
from acne_detection import AcneDetector
from age_gender_estimation import AgeGenderEstimator
from dark_eye_detection import DarkEyeDetector
from facial_landmark import FacialLandmark
from wrinkles_detection import WrinklesDetector
from recommendation_system import ProductRecommendation
from sallowness_detection import get_sallowness_score
from os.path import join
import pandas as pd
import config
import json
import random
import time
import imutils
import cv2

#PLEASE SET THINGS IN CONFIG.PY
PRODUCT_CATALOGUE_CSV_PATH = config.PRODUCT_CATALOGUE_CSV_PATH
WEB_DIR = config.WEB_DIR
WEB_ADDRESS = config.WEB_ADDRESS
MAIN_IMAGE_WIDTH = config.MAIN_IMAGE_WIDTH
SCORE_STANDARD = config.SCORE_STANDARD
MAX_ACNE_SCORE = config.MAX_ACNE_SCORE
MAX_WRINKLES_SCORE = config.MAX_WRINKLES_SCORE
MAX_CROWS_FEET_SCORE = config.MAX_CROWS_FEET_SCORE
MAX_DARK_EYE_SCORE = config.MAX_DARK_EYE_SCORE
MAX_SALLOWNESS_SCORE = config.MAX_SALLOWNESS_SCORE
EACH_ISSUE_N_RECOMM = config.EACH_ISSUE_N_RECOMM
FULL_N_RECOMM = config.FULL_N_RECOMM
ISSUES_LIST = config.FACIAL_ISSUES_KEYS
CONCERNS_NAME_MAPPING = config.CONCERNS_NAME_MAPPING
AVAILABLE_CONCERNS = config.AVAILABLE_CONCERNS
def standardize_score(score, from_standard, to_standard = SCORE_STANDARD):
    return (score/from_standard)*to_standard

#this function might need modification depending on the server setup
def saveImage(image, file_name, prefix,  directory = WEB_DIR, address = WEB_ADDRESS):
    rev_file_name = str(prefix) + "_" + file_name
    path = join(directory, rev_file_name)
    cv2.imwrite(path, image)
    
    return WEB_ADDRESS + "/" + rev_file_name

#identifier is a parameter that will appended to the processed images to differentiate between users. If unique_id is None, then the program will generate random ID
#questionnaire is string in json format
#image is an OPENCV2 IMAGE OBJECT from CV2.imread('path'). Since the cv2.imread behavior is to read image in "BGR" format, then pls read the image using opencv2 in the API code.

#questionnaire should have these keys: age, skin_type, allergies, price, concerns, preferences 
class SkinCareAdvisor():
    def __init__(self, questionnaire, image, identifier=None):
        if not identifier:
            identifier = random.getrandbits(64)
        self.identifier = identifier
        self.prod_cat = pd.read_csv(PRODUCT_CATALOGUE_CSV_PATH)
        facial_landmark = FacialLandmark(image)
        
        age_gender = AgeGenderEstimator(facial_landmark)
        dark_eyes = DarkEyeDetector(facial_landmark)
        acne = AcneDetector(facial_landmark)
        wrinkles = WrinklesDetector(facial_landmark)
        wrinkles.complete_init()
        
        #Take care of scoring matters first
        #acne
        acne_overall = acne.get_overall_score()
        self.acne_overall = standardize_score(acne_overall, MAX_ACNE_SCORE)
        self.acne_fh = standardize_score(acne.get_forehead_score(), MAX_ACNE_SCORE)
        acne_cheek_score = acne.get_cheeks_score()
        self.acne_rc = standardize_score(acne_cheek_score[0], MAX_ACNE_SCORE)
        self.acne_lc = standardize_score(acne_cheek_score[1], MAX_ACNE_SCORE)
        self.acne_ch = standardize_score(acne.get_chin_score(), MAX_ACNE_SCORE)
        
        #wrinkles
        wrinkles_overall = wrinkles.overall_score
        self.wrinkles_overall = standardize_score(wrinkles_overall, MAX_WRINKLES_SCORE)
        self.wrinkles_fh = standardize_score(wrinkles.fh_score, MAX_WRINKLES_SCORE)
        self.wrinkles_rnl = standardize_score(wrinkles.r_nl_score, MAX_WRINKLES_SCORE)
        self.wrinkles_lnl = standardize_score(wrinkles.l_nl_score, MAX_WRINKLES_SCORE)
        self.wrinkles_rbe = standardize_score(wrinkles.r_be_score, MAX_WRINKLES_SCORE)
        self.wrinkles_lbe = standardize_score(wrinkles.l_be_score, MAX_WRINKLES_SCORE)
        
        #crows feet
        crows_feet_overall = wrinkles.cf_score
        self.crows_feet_overall = standardize_score(crows_feet_overall, MAX_CROWS_FEET_SCORE)
        self.crows_feet_r = standardize_score(wrinkles.r_cf_score, MAX_CROWS_FEET_SCORE)
        self.crows_feet_l = standardize_score(wrinkles.l_cf_score, MAX_CROWS_FEET_SCORE)
        
        #dark_eye
        dark_eye_overall = dark_eyes.get_score()
        self.dark_eye_overall = standardize_score(dark_eye_overall, MAX_DARK_EYE_SCORE)
        
        #sallowness
        real_age = int(questionnaire["age"])
        predicted_age = age_gender.predict_age_gender()[0]
        sallowness_overall = get_sallowness_score(real_age,predicted_age)
        self.sallowness_overall = standardize_score(sallowness_overall, MAX_SALLOWNESS_SCORE)
        
        
        #Take care of the images
        resized_image = imutils.resize(image, width=MAIN_IMAGE_WIDTH)
        self.full_image = saveImage(resized_image, "full_image.jpg", self.identifier)
        
        #wrinkles
        self.wrinkles_fh_image = saveImage(wrinkles.fh_image, "wrinkles_fh_image.jpg", self.identifier)
        self.wrinkles_rnl_image = saveImage(wrinkles.r_nl_image, "wrinkles_rnl_image.jpg", self.identifier)
        self.wrinkles_lnl_image = saveImage(wrinkles.l_nl_image, "wrinkles_lnl_image.jpg", self.identifier)
        self.wrinkles_rbe_image = saveImage(wrinkles.r_be_image, "wrinkles_rbe_image.jpg", self.identifier)
        self.wrinkles_lbe_image = saveImage(wrinkles.l_be_image, "wrinkles_lbe_image.jpg", self.identifier)
        
        #crows_feet
        self.crows_feet_r_image = saveImage(wrinkles.r_cf_image, "crows_feet_r_image.jpg", self.identifier)
        self.crows_feet_l_image = saveImage(wrinkles.l_cf_image, "crows_feet_l_image.jpg", self.identifier)
        
        #Take care of the recommendation
        main_input = {
            'acne': acne_overall,
            'wrinkles': wrinkles_overall,
            'crows_feet': crows_feet_overall,
            'dark_eye': dark_eye_overall,
            'sallowness': sallowness_overall,
            'skin_type': questionnaire['skin_type'],
            'allergies': questionnaire['allergies'],
            'price': questionnaire['price'],
            'concerns': questionnaire['concerns'],
            'preferences': questionnaire['preferences']
        }
        prod_rec = ProductRecommendation(self.prod_cat, main_input)
        recom_object = prod_rec.get_default_recommendation()
        
        #full_recom
        full_recommendations = {}
        if 'all_products' in recom_object:
            recom_df = recom_object['all_products']
            counter = 0
            for idx, row in recom_df.iterrows():
                if FULL_N_RECOMM == counter:
                    break
                full_recommendations[idx] = {
                    "title":row['name'],
                    "issue": row['Label'],
                    "image":{
                        "uri": row['image_url']
                    },
                    "price":row['price'],
                    "rating":row['rating'],
                    "likes":row['likes'],
                    "description":row['description'],
                }
        self.full_recommendations = json.dumps(full_recommendations)
        
        
        #Acne
        self.acne_recommendation = self._get_recom_json(recom_object, 'acne','Cleanser', 'Acne')
        #Wrinkles
        self.wrinkles_recommendation = self._get_recom_json(recom_object, 'wrinkles', 'Treatment', 'Wrinkles')
        #crows_feet
        self.crows_feet_recommendation = self._get_recom_json(recom_object, 'crows_feet','Treatment', "Crow's Feet")
        #dark_eye
        self.dark_eye_recommendation = self._get_recom_json(recom_object, 'dark_eye','Eye cream', 'Dark Eye Circle')
        #sallowness
        self.sallowness_recommendation = self._get_recom_json(recom_object, 'sallowness','Face Mask', 'Sallowness')
        
        #taking care of concern
        self.concerns = self._extract_concerns_json(recom_object)
        
    def _get_recom_json(self, recom_object, key, label, issue_name, limit=EACH_ISSUE_N_RECOMM):
        tmp = {}
        if key in recom_object:
            if label:
                recom_df = recom_object[key][label]
            else:
                recom_df = recom_object[key]
            counter = 0
            for idx, row in recom_df.iterrows():
                if limit == counter:
                    break
                tmp[idx] = {
                    "title":row['name'],
                    "issue": issue_name,
                    "image":{
                        "uri": row['image_url']
                    },
                    "price":row['price'],
                    "rating":row['rating'],
                    "likes":row['likes'],
                    "description":row['description'],
                }
        return json.dumps(tmp)
    
    def _extract_concerns_json(self, recom_object):
        tmp = {}
        for key in recom_object:
            if (key not in ISSUES_LIST) and (key in AVAILABLE_CONCERNS):
                concern_name = CONCERNS_NAME_MAPPING[key]
                tmp[concern_name] = {}
                tmp[concern_name]["recommendation"] = {}
                recom_df = recom_object[key]
                counter = 0
                for idx, row in recom_df.iterrows():
                    if EACH_ISSUE_N_RECOMM == counter:
                        break
                    tmp[concern_name]["recommendation"][idx] = {
                        "title":row['name'],
                        "issue": concern_name,
                        "image":{
                            "uri": row['image_url']
                        },
                        "price":row['price'],
                        "rating":row['rating'],
                        "likes":row['likes'],
                        "description":row['description'],
                    }
        return json.dumps(tmp)

In [8]:
import cv2
import time

q = {
    'age' : 21,
    'skin_type': "normal",
    'allergies': None,
    'price': None,
    'concerns': ["acne", "dark_eye", "oiliness"],
    'preferences': None
}
image = cv2.imread("./test_images/image2.jpg")
start_time = time.time()
sca = SkinCareAdvisor(q, image)
print("Time full pipeline", time.time() - start_time, "second(s)")

c:\users\denis\anaconda3\envs\fyp\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\denis\anaconda3\envs\fyp\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Time full pipeline 6.264191389083862 second(s)


In [9]:
sca.concerns

'{"Oiliness": {"recommendation": {"0": {"title": "The Deep Cleanse Exfoliating Cleanser", "issue": "Oiliness", "image": {"uri": "https://www.sephora.com/productimages/sku/s2315331-main-zoom.jpg"}, "price": 38.0, "rating": 4.3, "likes": 94300, "description": "A daily gel cleanser with natural fruit exfoliant that thoroughly but gently lifts impurities and unclogs pores while leaving skin soft and hydrated."}, "1": {"title": "Superfood Antioxidant Cleanser", "issue": "Oiliness", "image": {"uri": "https://www.sephora.com/productimages/sku/s1863588-main-zoom.jpg"}, "price": 36.0, "rating": 4.3, "likes": 79100, "description": "A daily green juice cleanse for your face with cold-pressed antioxidants to remove makeup and prevent build-up in pores, while keeping skin\\u2019s pH balanced."}, "2": {"title": "Beste\\u2122 No. 9 Jelly Cleanser", "issue": "Oiliness", "image": {"uri": "https://www.sephora.com/productimages/sku/s2022598-main-zoom.jpg"}, "price": 32.0, "rating": 4.0, "likes": 68100, "

In [6]:
sca.full_recommendations

'{"0": {"title": "Glycolic Acid 7% Toning Solution", "issue": "Cleanser", "image": {"uri": "https://www.sephora.com/productimages/sku/s2031508-main-zoom.jpg"}, "price": 8.699999999999998, "rating": 4.5, "likes": 172100, "description": "An exfoliating toning solution with seven percent glycolic acid, amino acids, aloe vera, ginseng, and tasmanian pepperberry."}, "1": {"title": "Protini\\u2122 Polypeptide Moisturizer", "issue": "Moisturizer", "image": {"uri": "https://www.sephora.com/productimages/sku/s2025633-main-zoom.jpg"}, "price": 68.0, "rating": 4.0, "likes": 218900, "description": "A protein moisturizer that combines signal peptides, growth factors, amino acids, and pygmy waterlily to improve the look of skin\\u2019s tone, texture, and firmness."}, "2": {"title": "Truth Serum\\u00ae", "issue": "Treatment", "image": {"uri": "https://www.sephora.com/productimages/sku/s1910470-main-zoom.jpg"}, "price": 50.0, "rating": 4.2, "likes": 202200, "description": "A powerful antiaging serum f

In [13]:
sca.dark_eye_recommendation

'{"0": {"title": "Banana Bright Eye Cr\\u00e8me", "issue": "Dark Eye Circle", "image": {"uri": "https://www.sephora.com/productimages/sku/s2018984-main-zoom.jpg"}, "price": 39.0, "rating": 4.0, "likes": 177400, "description": "A brightening, vitamin C-rich eye cr\\u00e8me that targets signs of aging, reducing the look of dark circles while improving concealer application and wear\\u2014inspired by makeup artist secret, banana powder."}, "1": {"title": "All About Eyes\\u2122 Rich Eye Cream", "issue": "Dark Eye Circle", "image": {"uri": "https://www.sephora.com/productimages/sku/s1246693-main-zoom.jpg"}, "price": 54.0, "rating": 4.1, "likes": 53600, "description": "A rich eye cream that diminishes the look of under-eye circles, shadows, puffs, and fine lines."}, "2": {"title": "Eye Duty Triple Remedy: Brighten, Depuff and Smooth", "issue": "Dark Eye Circle", "image": {"uri": "https://www.sephora.com/productimages/sku/s1600162-main-zoom.jpg"}, "price": 36.0, "rating": 3.9, "likes": 50000,

In [4]:
sca.full_recommendations['all_products'][["rating", "likes", "price"]]

,rating,likes,price
0,4.5,172100,8.7
1,4.0,218900,68.0
2,4.2,202200,50.0
3,4.7,162300,7.0
4,4.5,172800,24.0
...,...,...,...
130,4.2,13600,153.0
131,4.4,4700,155.0
132,4.3,12300,235.0
133,3.6,20600,225.0


In [8]:
sca.full_recommendations['all_products']

,Label,URL,brand,name,price,rating,description,skin_type,skincare_concerns,ingredients,...,dullness,dark_spots,pores,redness,acne,uneven_skin_tone,sallowness,class_specified,image_url,scores
0,Cleanser,https://www.sephora.com/product/the-ordinary-d...,THE ORDINARY,Glycolic Acid 7% Toning Solution,8.7,4.5,An exfoliating toning solution with seven perc...,"[1, 1, 1, 1, 1]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","water, glycolic acid, rosa damascena flower wa...",...,1,0,0,0,0,0,0,dullness,https://www.sephora.com/productimages/sku/s203...,0.892767
1,Moisturizer,https://www.sephora.com/product/protini-tm-pol...,DRUNK ELEPHANT,Protini™ Polypeptide Moisturizer,68.0,4.0,A protein moisturizer that combines signal pep...,"[1, 1, 1, 1, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 1]","water/aqua/eau, dicaprylyl carbonate, glycerin...",...,1,0,0,0,0,0,1,loss of firmness,https://www.sephora.com/productimages/sku/s202...,0.892128
2,Treatment,https://www.sephora.com/product/truth-serum-P4...,OLEHENRIKSEN,Truth Serum®,50.0,4.2,A powerful antiaging serum formulated with vit...,"[1, 1, 1, 1, 1]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 1]","water, sodium ascorbyl phosphate, calcium asco...",...,1,0,0,0,0,0,1,dullness,https://www.sephora.com/productimages/sku/s191...,0.890745
3,Treatment,https://www.sephora.com/product/aha-30-bha-2-p...,THE ORDINARY,AHA 30% + BHA 2% Peeling Solution,7.0,4.7,An exfoliating solution to help fight visible ...,"[1, 1, 1, 0, 0]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]","glycolic acid, water, aloe barbadensis leaf wa...",...,1,0,0,0,0,0,0,dullness,https://www.sephora.com/productimages/sku/s221...,0.888545
4,Cleanser,https://www.sephora.com/product/purity-made-si...,PHILOSOPHY,Purity Made Simple Cleanser,24.0,4.5,"A face wash for easy, one-step cleansing—Purit...","[1, 1, 1, 1, 0]","[1, 0, 0, 1, 0, 1, 0, 0, 0, 0]","water, sodium lauroamphoacetate, sodium tridec...",...,1,0,1,0,0,0,0,pores,https://www.sephora.com/productimages/sku/s407...,0.880865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Face Mask,https://www.sephora.com/product/creme-ancienne...,FRESH,Crème Ancienne® Ultimate Nourishing Honey Mask,153.0,4.2,An intensely nourishing face mask with almost ...,"[1, 1, 1, 1, 1]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 1]","mel (honey), propylene glycol dicaprylate/dica...",...,0,0,0,0,0,0,1,loss of firmness,https://www.sephora.com/productimages/sku/s155...,0.501532
131,Face Mask,https://www.sephora.com/product/treatment-loti...,LA MER,Treatment Lotion Hydrating Mask,155.0,4.4,"A mask that visibly plumps, hydrates, and nurt...","[1, 1, 1, 1, 0]","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0]","water\aqua\eau, algae (seaweed) extract, glyce...",...,1,0,0,0,0,0,0,dullness,https://www.sephora.com/productimages/sku/s213...,0.495600
132,Moisturizer,https://www.sephora.com/product/r-n-a-power-ra...,SK-II,R.N.A. POWER Anti-Aging Face Cream,235.0,4.3,A bestselling moisturizing cream to supercharg...,"[1, 0, 0, 1, 1]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 1]","water, glycerin, galactomyces ferment filtrate...",...,1,0,0,0,0,0,1,loss of firmness,https://www.sephora.com/productimages/sku/s178...,0.434667
133,Eye cream,https://www.sephora.com/product/la-mer-the-eye...,LA MER,The Eye Concentrate,225.0,3.6,A hydrating eye cream that visibly diminishes ...,"[1, 1, 1, 1, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","algae (seaweed) extract, water\aqua\eau, cyclo...",...,0,0,0,0,0,0,0,dark circles,https://www.sephora.com/productimages/sku/s234...,0.421448


issue -> score, image, recommendation
score -> overall, etc.
image -> etc.
recommendation: 